In [ ]:
# %load ../notebooks/init.ipy
%reload_ext autoreload
%autoreload 2

# Builtin packages
from importlib import reload
import logging
import os
from pathlib import Path
import sys
import warnings

# standard secondary packages
import astropy as ap
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.stats
import tqdm.notebook as tqdm

# development packages
import kalepy as kale
import kalepy.utils
import kalepy.plot

# --- Holodeck ----
import holodeck as holo
import holodeck.sam
from holodeck import cosmo, utils, plot
from holodeck.constants import MSOL, PC, YR, MPC, GYR
import holodeck.gravwaves
import holodeck.evolution
import holodeck.population

# Silence annoying numpy errors
np.seterr(divide='ignore', invalid='ignore', over='ignore')
warnings.filterwarnings("ignore", category=UserWarning)

# Plotting settings
mpl.rc('font', **{'family': 'serif', 'sans-serif': ['Times'], 'size': 15})
mpl.rc('lines', solid_capstyle='round')
mpl.rc('mathtext', fontset='cm')
mpl.style.use('default')   # avoid dark backgrounds from dark theme vscode
plt.rcParams.update({'grid.alpha': 0.5})

# Load log and set logging level
log = holo.log
log.setLevel(logging.INFO)

In [ ]:
from holodeck.constants import NWTG, SPLC, GPC

LABEL_FREQ_YRS = 'Frequency $[\mathrm{yr}^{-1}]$'
LABEL_STRAIN_AMP = 'GW Strain Amplitude'
LABEL_DIST_LUM_MPC = 'Luminosity Distance $[\mathrm{Mpc}]$'
LABEL_NDENS_MPC3 = 'Number Density $[\mathrm{cMpc}^{-3}]$'
LABEL_MASS_TOTAL = 'Total Mass $[M_\odot]$'

def _twin_redz_from_dlum_mpc(ax, zcutoff=0.01):
    yticks = np.array(ax.get_yticks())
    ytick_labels = ax.get_yticklabels()

    zticks = cosmo.dlum_to_z(yticks * MPC)
    ztick_labels = [f"{zz:.2f}" if zz > zcutoff else "" for zz in zticks]

    tw = ax.twinx()
    tw.set_yticks(yticks)
    tw.set(yscale='log', ylim=ax.get_ylim(), ylabel='Redshift')
    tw.set_yticklabels(ztick_labels)
    return tw


# Load Data

In [ ]:
path = Path("/Users/lzkelley/Programs/nanograv/holodeck/data/caitlin-witt_2022-08-05_final-curves/")
fname = "CW_fixCRN_UL_final.txt"
path = path.joinpath(fname)
data = np.loadtxt(path)

ul_freq = data[:, 0]    # hz
ul_strain = data[:, 1]      # hs?  hc?
ul_strain_err = data[:, 2]
fig, ax = plot.figax()

# freq = ul_freq
# freq = ul_freq * YR
NUM = 21
UNITS = 1/YR

num = (NUM + 1) // 2
assert num >= 2

ax.plot(ul_freq/UNITS, ul_strain, lw=1.5, alpha=0.5, color='k')
ax.errorbar(ul_freq/UNITS, ul_strain, yerr=ul_strain_err, ls='none', alpha=0.5, lw=1.0, color='r')

imin = np.argmin(ul_strain)
xmin = ul_freq[imin]
ymin = ul_strain[imin]
print(f"{ymin:.2e} at {UNITS/xmin:.2f} yr ({xmin/UNITS:.2e} 1/yr = {xmin:.2e} Hz)")
ax.axvline(xmin/UNITS, color='r', alpha=0.25)
ax.axhline(ymin, color='r', alpha=0.25)

xr = kale.utils.spacing([xmin, 100*xmin], scale='log', num=num)
yr = ymin * np.power(xr/xmin, 5.0/6.0)
ax.plot(xr/UNITS, yr, 'b--', alpha=0.25)

# powers = [1.0, 5.0/6.0, 2.0/3.0, 0.5]
# for pp in powers:
#     yr = ymin * np.power(xr/xmin, pp)
#     ax.plot(xr, yr, ls='--', alpha=0.5, label=f'{pp:.4f}')

xl = kale.utils.spacing([xmin, xmin/10.0], scale='log', num=num)
yl = ymin * np.power(xl/xmin, -1.0)
ax.plot(xl/UNITS, yl, 'b--', alpha=0.25)
# powers = [3.0/2.0, 1.0, 5.0/6.0]
# for pp in powers:
#     yl = ymin * np.power(xl/xmin, -pp)
#     ax.plot(xl, yl, ls='--', alpha=0.5, label=f'{-pp:.4f}')


plot._twin_hz(ax, fs=10)
ax.set(xlabel=LABEL_FREQ_YRS, ylabel=LABEL_STRAIN_AMP)

# ax.legend()
plt.show()


# freqs = np.concatenate([xl, xr])
# strains = np.concatenate([yl, yr])
# _, iq = np.unique(freqs, return_index=True)
# freqs = freqs[iq]
# strains = strains[iq]

freqs = ul_freq
strains = ul_strain

# Calculate Luminosity-Distance

For a circular binary, sky and polarization averaged, the GW strain amplitude is:
$$h_s = \frac{8}{10^{1/2}} \frac{(G \mathcal{M})^{5/3}}{c^4 d_c} (2\pi f_r)^{2/3},$$
where $\mathcal{M}$ is the rest-frame chirp-mass, $d_c$ is the comoving distance, and $f_r$ is the rest-frame *orbital* frequency.  This can also be expressed in terms of observer-frame quantities as,
$$h_s = \frac{8}{10^{1/2}} \frac{(G \mathcal{M_o})^{5/3}}{c^4 d_L} (2\pi f_o)^{2/3},$$
where $d_L$ is the luminosity-distance, and $\mathcal{M}_o = \mathcal{M} (1+z)$ and $f_o = f_r/(1+z)$ are the observer-frame chirp-mass and *orbital* frequency respectively.

In [ ]:
def _dist_lum_from_strain_amp_and_mchirp_obs(hs, mchirp_obs, fobs_orb):
    fterm = np.power(2*np.pi*fobs_orb, 2.0/3.0)
    mterm = np.power(NWTG*mchirp_obs, 5.0/3.0) / SPLC**4
    dist_lum = (8 / np.sqrt(10)) * mterm * fterm / hs
    return dist_lum

def _mchirp_obs_to_mchirp_rst(dist_lum, mchirp_obs, fobs_orb):
    redz = cosmo.dlum_to_z(dist_lum)
    zp1 = redz + 1.0
    mchirp_rst = mchirp_obs / zp1
    frst_orb = fobs_orb * zp1
    return mchirp_rst, frst_orb

def _strain_from_obs(dist_lum, mchirp_obs, fobs_orb):
    fterm = np.power(2*np.pi*fobs_orb, 2.0/3.0)
    mterm = np.power(NWTG*mchirp_obs, 5.0/3.0) / SPLC**4
    hs = (8 / np.sqrt(10)) * mterm * fterm / dist_lum
    return hs

def dist_from_strain_amp(hs, mchirp_rst, fobs_gw):
    """Calculate luminosity distance producing the given GW Strain-Amplitude, from the given binary properties.
    
    Parameters
    ----------
    hs : GW strain-amplitude
    mchirp_rst : rest-frame chirp-mass [gram]
    fobs_gw : observer-frame GW-frequency [1/sec]
    
    Returns
    -------
    dist_lum : luminosity-distance [cm]
    dist_com : comoving-distance [cm]
    redz : redshift
    
    """
    # convert from GW to orbital frequency (assuming circular orbit)
    fobs_orb = fobs_gw / 2.0
    # approximate the distance assuming z~0 and thus mc_obs ~ mc_rst
    dist = _dist_lum_from_strain_amp_and_mchirp_obs(hs, mchirp_rst, fobs_orb)

    # Function that returns the difference between actual and guessed strain, given a luminosity-distance
    def func(dl, verbose=False):
        # get redshift from luminosity-distance
        zz = cosmo.dlum_to_z(dl)
        # if zz > 10.0:
        #     raise ValueError
        # convert from rest-frame chirp-mass to observer frame
        mc_obs = mchirp_rst * (1.0 + zz)
        # calculate GW strain-amp for these values
        hs_test = _strain_from_obs(dl, mc_obs, fobs_orb)
        if verbose:
            print(f"{dl=} {zz=} {mc_obs=} {hs_test=}")

        # return fractional error
        return (hs - hs_test) / hs
    
    dist_lum = sp.optimize.newton(func, dist, rtol=1e-2, maxiter=int(50))
    # try:
    #     dist_lum = sp.optimize.newton(func, dist, rtol=1e-2, maxiter=int(50))
    # except:
    #     print(f"{hs=:.4e}, {mchirp_rst/MSOL=:.4e}, {fobs_gw*YR=:.4e}")
    #     print(f"{dist/MPC=:.4e} {func(dist, verbose=True)=:.4e}")
    #     dist_lum = sp.optimize.newton(lambda xx: func(xx, True), dist, rtol=1e-2, maxiter=int(1e3))
    #     raise
    
    redz = cosmo.dlum_to_z(dist_lum)
    dist_com = cosmo.comoving_distance(redz).cgs.value
    # err = func(dist_lum)
    return dist_lum, dist_com, redz


warnings.filterwarnings("ignore", category=RuntimeWarning, message="RMS of *")


In [ ]:
fig, ax = plot.figax()
mchirp_list = [8.0, 8.5, 9.0, 9.5]
for mc in mchirp_list[::-1]:
    mc = MSOL * (10.0 ** mc)
    dl_limit, *_ = dist_from_strain_amp(strains, mc, freqs)
    ax.plot(freqs*YR, dl_limit/MPC, label=f'{np.log10(mc/MSOL):.1f}')

plot._twin_hz(ax, fs=10)
ax.set(xlabel=LABEL_FREQ_YRS, ylabel=LABEL_DIST_LUM_MPC, ylim=[1, 1e3])
ax.legend(title='Chirp Mass $[\log_{10}(\mathcal{M}/\mathrm{M}_\odot)]$')

plt.show()

In [ ]:
fig, ax = plot.figax(top=0.78, bottom=0.15)
mrat_list = [0.1, 0.3, 0.5]
cmap_list = ['Reds', 'Blues', 'Purples']
mtot_list = [8.5, 9.0, 9.5, 10.0]
# lines_list = ['--', '-', '--', '-']
lines_list = None
handles_mrat = []
labels_mrat = []
handles_mtot = []
labels_mtot = []
last_mtot = len(mtot_list) - 1
for ii, (mrat, cmap) in enumerate(zip(mrat_list, cmap_list)):
    smap = plot.smap(mtot_list, cmap=cmap, left=0.4, right=0.8)
    colors = smap.to_rgba(mtot_list)
    for jj, (_mt, col) in enumerate(zip(mtot_list, colors)):
        mtot = MSOL * (10.0 ** _mt)
        mc = utils.chirp_mass(utils.m1m2_from_mtmr(mtot, mrat))
        
        dl_limit, *_ = dist_from_strain_amp(strains, mc, freqs)
        label = f"{_mt:.2f}, {mrat:.2f}"
        ls = '-' if lines_list is None else lines_list[jj]
        hh, = ax.plot(freqs*YR, dl_limit/MPC, color=col, ls=ls)

        if ii == 2:
            labels_mtot.append(f"{_mt:.1f}")
            handles_mtot.append(hh)

        if jj == 1:
            labels_mrat.append(f"{mrat:.1f}")
            handles_mrat.append(hh)

plot._twin_hz(ax, fs=10)

ax.set(xlabel=LABEL_FREQ_YRS, ylabel=LABEL_DIST_LUM_MPC, ylim=[1.0, 1e3])
leg = ax.legend(
    handles_mtot, labels_mtot, bbox_transform=fig.transFigure, bbox_to_anchor=(1.0, 1.0),
    title='Total Mass $[\log_{10}(M/\mathrm{M}_\odot)]$', loc='upper right', ncol=len(labels_mtot)
)
ax.legend(
    handles_mrat, labels_mrat, bbox_transform=fig.transFigure, bbox_to_anchor=(0.01, 1.0),
    title='Mass Ratio', loc='upper left', ncol=len(labels_mrat)
)
ax.add_artist(leg)

_twin_redz_from_dlum_mpc(ax, zcutoff=0.01)
plt.show()
# fig.savefig('test.png')

# Calculate Number-Density *of a given binary configuration*

If we place a limit, such that there are no sources (of given binary parameters) within a comoving distance $d_c$, we can say the local density is less than $n_c = 1/V_c = [(4/3) \pi d_c^3]^{-1}$.  To consider this as a limit on the average density in some volume, that is relatively-local but larger than the explicitly measured volume, there should be some additional pre-factor to account for the confidence of having a source within this volume based on Poisson (or similar) distributions of sources.

To be more precise, define an expected number of events $\Lambda = n_c V_c$, then the likelihood for $N$ detections in the volume $V_c$ is, $P_N(\Lambda) = \frac{\Lambda^N e^{-\Lambda}}{N!}$.  For no detections, the likelihood is $P_0(\Lambda) = e^{- \Lambda}$.  To find an upper-limit on the occurrence rate, $\Lambda_{ul}$, we must integrate from that limit to infinity, such that the result matches our desired confidence level $p_0$ (e.g.~if $p_0=0.95$, then there is a $95\%$ chance that the rate is below this upper-limit): $F_{ul}(\Lambda_{ul}) = \int_{\Lambda_{ul}}^\infty e^{-\Lambda} d\Lambda = 1 - p_0$.  Or, converting this back to the dimensional volume density, we find,
$$n_{ul} = \frac{-\ln (1-p_0)}{V_c}.$$

Plot $95\%$ upper-limits on number density of sources at some particular chirp-masses.

In [ ]:
def ndens_from_strain_amp(hs, mchirp_rst, fobs_gw, conf=0.95):
    assert 0.0 < conf < 1.0
    try:
        dist_lum, dist_com, redz = dist_from_strain_amp(hs, mchirp_rst, fobs_gw)
    except Exception as err:
        # print(dist_lum)
        log.error(err)
        dist_lum = 1e5 * MPC
        redz = cosmo.dlum_to_z(dist_lum)
        dist_com = cosmo.comoving_distance(redz).cgs.value
        # print(dist_lum, redz, dist_com)
        
    vol_com = (4.0/3.0) * np.pi * (dist_com/MPC) ** 3
    conf_factor = - np.log(1.0 - conf)
    ndens_com_mpc3 = conf_factor / vol_com
    return ndens_com_mpc3
    
fig, ax = plot.figax()
mchirp_list = [8.0, 8.5, 9.0, 9.5]
for mc in mchirp_list:
    mc = MSOL * (10.0 ** mc)
    ndens_limit = ndens_from_strain_amp(strains, mc, freqs)
    ax.plot(freqs*YR, ndens_limit, label=f'{np.log10(mc/MSOL):.1f}')

plot._twin_hz(ax, fs=10)
ax.set(xlabel=LABEL_FREQ_YRS, ylabel=LABEL_NDENS_MPC3, ylim=[1e-9, 1e3])
ax.legend(title='Chirp Mass $[\log_{10}(\mathcal{M}/\mathrm{M}_\odot)]$')

plt.show()

Plot $95\%$ upper-limits on number density of sources at some particular total-masses and mass-ratios.

In [ ]:
fig, ax = plot.figax(top=0.78, bottom=0.15, left=0.08, right=0.98)
mrat_list = [0.1, 0.3, 0.5]
cmap_list = ['Reds', 'Blues', 'Purples']
mtot_list = [8.5, 9.0, 9.5, 10.0]
# lines_list = ['--', '-', '--', '-']
lines_list = None
handles_mrat = []
labels_mrat = []
handles_mtot = []
labels_mtot = []
last_mtot = len(mtot_list) - 1
for ii, (mrat, cmap) in enumerate(zip(mrat_list, cmap_list)):
    smap = plot.smap(mtot_list, cmap=cmap, left=0.4, right=0.8)
    colors = smap.to_rgba(mtot_list)
    for jj, (_mt, col) in enumerate(zip(mtot_list, colors)):
        mtot = MSOL * (10.0 ** _mt)
        mc = utils.chirp_mass(utils.m1m2_from_mtmr(mtot, mrat))
        
        ndens_limit = ndens_from_strain_amp(strains, mc, freqs)
        label = f"{_mt:.2f}, {mrat:.2f}"
        ls = '-' if lines_list is None else lines_list[jj]
        hh, = ax.plot(freqs*YR, ndens_limit, color=col, ls=ls)

        if ii == 2:
            labels_mtot.append(f"{_mt:.1f}")
            handles_mtot.append(hh)

        if jj == 1:
            labels_mrat.append(f"{mrat:.1f}")
            handles_mrat.append(hh)

plot._twin_hz(ax, fs=10)

ax.set(xlabel=LABEL_FREQ_YRS, ylabel=LABEL_NDENS_MPC3, ylim=[1.0e-9, 1e2])
leg = ax.legend(
    handles_mtot, labels_mtot, bbox_transform=fig.transFigure, bbox_to_anchor=(1.0, 1.0),
    title='Total Mass $[\log_{10}(M/\mathrm{M}_\odot)]$', loc='upper right', ncol=len(labels_mtot)
)
ax.legend(
    handles_mrat, labels_mrat, bbox_transform=fig.transFigure, bbox_to_anchor=(0.01, 1.0),
    title='Mass Ratio', loc='upper left', ncol=len(labels_mrat)
)
ax.add_artist(leg)

plt.show()
fig.savefig('ndens-constraints_witt.png')

# Calculate Number-Density of *all binaries, based on some population model*

In [ ]:
fobs_orb = freqs / 2.0   # convert from GW to orbital frequency (assume circular orbits)
pop_sam = holo.sam.Semi_Analytic_Model()
# time = 5 * GYR
# hard = holo.evolution.Fixed_Time.from_sam(pop_sam, time)
hard = holo.evolution.Hard_GW()

In [ ]:
# `edges` is (4,) of array giving the bin-edges of total-mass, mass-ratio, redshift, freq-obs-orbital
# `d4_num_sam` is shaped (M, Q, Z, F) where each dimension length is the number of bin-edges minus 1
#     this is `d^4 N / [dlog10(M) dq dz dln(f_r)]`
edges, d4_num_sam = pop_sam.dynamic_binary_number(hard, fobs=fobs_orb)

In [ ]:
# Integrate over each bin to find the number of binaries in each bin.  Units of number (dimensionless).
sam_number = holo.utils._integrate_grid_differential_number(edges, d4_num_sam, freq=True)

In [ ]:
# Convert back to volume-density of binaries in each bin, i.e. [Mpc^-3]
dvol = np.diff(cosmo.comoving_volume(edges[2])).to('Mpc3').value
sam_ndens = sam_number / dvol[np.newaxis, np.newaxis, :, np.newaxis]

In [ ]:
fbin = np.argmin(ul_strain)
print(f"{fbin=}")
ndens = sam_ndens[..., fbin]
imax = np.argmax(ndens, axis=-1)
assert np.all(imax == imax.flatten()[0])
imax = imax.flatten()[0]
ndens = ndens[..., imax]

mrat_list = [0.05, 0.1, 0.3, 0.5]

fig, ax = plot.figax(
    top=0.97, bottom=0.16, left=0.08, right=0.92,
    xlabel=LABEL_MASS_TOTAL, ylabel=LABEL_NDENS_MPC3
)
tw = ax.twinx()
tw.set(yscale='log', ylabel='Ratio (upper-limit / model | dotted)')
ax.set(ylim=[1e-12, 1e7])
tw.set(ylim=[1e6, 1e10])

mtot_centers = kale.utils.midpoints(edges[0], log=True)
xx = mtot_centers / MSOL

for mrat in mrat_list:
    qidx = np.searchsorted(edges[1], mrat)
    mrat = edges[1][qidx]
    
    # sum over all larger mass-ratios
    ndens_array = np.sum(ndens[:, qidx:], axis=1)
    # cumulative sum over all larger masses, for each total-mass bin
    ndens_array = np.cumsum(ndens_array[::-1])[::-1]

    ndens_limit = np.zeros_like(ndens_array)
    for ii, mt in enumerate(mtot_centers):
        mc = holo.utils.chirp_mass(holo.utils.m1m2_from_mtmr(mt, mrat))
        # print(ii, f"{mc/MSOL:.2e}", fbin, freqs[fbin], strains[fbin])
        ndens_limit[ii] = ndens_from_strain_amp(strains[fbin], mc, freqs[fbin], conf=0.95)

    hh, = ax.plot(xx, ndens_array, label=f'{mrat:.2f}')
    cc = hh.get_color()
    ax.plot(xx, ndens_limit, color=cc, ls='--')

    tw.plot(xx, ndens_limit/ndens_array, color=cc, ls=':')

ax.legend(title='Mass Ratio')
plt.show()
fig.savefig('population-limit_witt.png')